In [1]:
import sys
import platform
from os.path import join, exists, abspath, dirname
from os import getcwd, makedirs
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm, colors
import scipy
from scipy.stats import ttest_1samp, pearsonr
from scipy.io import loadmat
from sklearn.metrics import r2_score
import seaborn as sns
import h5py

from tqdm import tqdm

In [2]:
import nibabel as nb
from nilearn import plotting, image
from nipype.interfaces import fsl

In [3]:
dir_current = getcwd().replace('\\','/')

tmp = dir_current.split('/')
idx = [ii for ii, s in enumerate(tmp) if s=='github'][0]

dir_git = '/'.join(tmp[:idx+1])
dir_git

'/Users/sungbeenpark/github'

In [4]:
dname = join(dir_git,'nitools')
sys.path.append(dname)
import nitools as nt

In [5]:
dname = join(dir_git,'SUITPy')
sys.path.append(dname)
import SUITPy as suit

In [6]:
dname = join(dir_git)
sys.path.append(dname)
import surfAnalysisPy as surf

In [7]:
dname = join(dir_git,'Functional_Fusion')
sys.path.append(dname)
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel

In [8]:
dname = join(dir_git,'AnatSearchlight')
sys.path.append(dname)
import AnatSearchlight.searchlight as sl

In [9]:
dname = abspath(join(dir_git,'PcmPy'))
sys.path.append(dname)
import PcmPy as pcm

In [10]:
dname = join(dir_git,'SeqSpatialSupp_fMRI')
sys.path.append(dname)
from SSS import deal_spm
from SSS import util as su
from SSS import plot as splt
from SSS import image as simage
from SSS import glmsingle as ssingle

---

In [11]:
hem = 'L'

In [12]:
list_sn = su.get_list_sn()

---

### Setup to run

To define a searchlight we need a:

- Pial Surface (individual)
- Gray Matter Mask (individual)
- Functional mask image (mask.nii from GLM) telling us which voxels are availabe for searchlight analysis

If we want to calculate the searchlight only for a part of a the surface, we can use a roi_specification (i.e. excluding the medial wall).

The searchlight can be specified to have a specific radius in mm (on the surface) or a specific number of voxels.

In [13]:
dir_output= join(su.get_dir_surf())
dir_output

'/Volumes/Diedrichsen_data$/data/SeqSpatialSupp_fMRI/surfaceWB'

In [17]:
for ii, sn in enumerate(tqdm(list_sn)):
    subj = 'S'+sn
    
    # These are the inndividual surfaces and the mask
    white, pial, mask = simage.get_WPM(subj=subj, glm=1, hem=hem)

    # Optionally, you can define a roi-mask for the searchlight region on the surface. Centers outside of the mask will be ignored.
    # Instead of a gifti-image you can also use a numpy array with the indices of the vertices that should be included in the searchlight.
    roi_mask = join(dir_git,'AnatSearchlight/examples/tpl-fs32k_hemi-%s_mask.label.gii'%hem)

    # Give it an appropriate structure name: Used for generating cifti-files for output
    if hem == 'L':
        mysearchlight = sl.SearchlightSurface('left_cortex')
    if hem == 'R':
        mysearchlight = sl.SearchlightSurface('right_cortex')
    
    # Here we defining
    maxradius = 100
    maxvoxels = 200
    
    mysearchlight.define(
        surfs=[pial,white],
        mask_img=mask,
        roi=roi_mask,
        maxradius=maxradius,
        maxvoxels=maxvoxels,
        verbose=False
    )

    fname = join(dir_output,subj,'%s.searchlight_surface.nvoxel=%d.h5'%(subj,maxvoxels))
    mysearchlight.save(fname)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [3:48:18<00:00, 1141.55s/it]


---